In [1]:
!pip install torch opencv-python matplotlib pyyaml

  Using cached torch-2.8.0-cp313-cp313-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 

In [2]:
!pip install numpy==1.26.4

  Using cached numpy-1.26.4-cp313-cp313-linux_x86_64.whl
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [1]:
import torch
torch.cuda.is_available()

False

In [2]:
# Convert .pth to sp.dat

# Load SuperPoint pretrained weights
model = torch.load('SuperPointPretrainedNetwork/superpoint_v1.pth', map_location='cpu')

# Save to sp.dat (RISG expects this format)
with open('RISG-image-matching/db/sp.dat', 'wb') as f:
    torch.save(model, f)

In [3]:
# Convert superglue_outdoor.pth to sg.dat

# Load SuperGlue weights
sg_model = torch.load('SuperGluePretrainedNetwork/models/weights/superglue_outdoor.pth', map_location='cpu')

# Save in RISG-expected format
with open('RISG-image-matching/db/sg.dat', 'wb') as f:
    torch.save(sg_model, f)

In [4]:
# Visualize the matches
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

def draw_matches(img0, img1, mkpts0, mkpts1):
    img0 = cv.cvtColor(img0, cv.COLOR_BGR2RGB)
    img1 = cv.cvtColor(img1, cv.COLOR_BGR2RGB)

    h1, w1 = img0.shape[:2]
    h2, w2 = img1.shape[:2]
    canvas = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
    canvas[:h1, :w1] = img0
    canvas[:h2, w1:] = img1

    mkpts0_int = np.round(mkpts0).astype(int)
    mkpts1_int = np.round(mkpts1).astype(int)

    for pt0, pt1 in zip(mkpts0_int, mkpts1_int):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = pt0
        x1, y1 = pt1
        cv.circle(canvas, (x0, y0), 3, color, -1)
        cv.circle(canvas, (x1 + w1, y1), 3, color, -1)
        cv.line(canvas, (x0, y0), (x1 + w1, y1), color, 1)

    plt.figure(figsize=(16, 8))
    plt.imshow(canvas)
    plt.axis('off')
    plt.title(f'{len(mkpts0)} matches')
    plt.show()

In [5]:
%cd RISG-image-matching/

/home/kali/Documents/personal/computer_vision/risg_image_matching/RISG-image-matching


In [6]:
import os
from risgmatching import RISGMatcher
import yaml
import json

def run_model(risg, img_filename0, img_filename1):
    img0 = cv.imread(img_filename0)
    img1 = cv.imread(img_filename1)
    # rotate is number of directions
    mkpts0, mkpts1, conf, main_dir = risg.match(img0, img1, nrotate = 5)
    return img0, img1, mkpts0, mkpts1, conf, main_dir

# Benchmark the matching performance
def create_benchmark(conf, img_num, threshold):
    conf = np.asarray(conf).reshape(-1)  # ensure 1-D array
    total = int(conf.size)

    if total == 0:
        return {
            "image_num": img_num,
            "threshold": float(threshold),
            "total_matches": 0,
            "mean_confidence": None,
            "strong_match_count": 0,
            "accuracy": 0.0,
            "best_matches": []
        }

    strong_mask = conf >= threshold
    strong_count = int(np.count_nonzero(strong_mask))
    accuracy = float(strong_count / total)

    # Indices of strong matches, sorted by confidence descending
    strong_indices = np.flatnonzero(strong_mask)
    sort_order = np.argsort(conf[strong_mask])[::-1]  # descending
    best_indices = strong_indices[sort_order]
    best_confidences = conf[best_indices]

    return {
        "image_num": img_num,
        "threshold": float(threshold),
        "total_matches": total,
        "mean_confidence": float(conf.mean()),
        "strong_match_count": strong_count,
        "accuracy": accuracy,
        "best_matches": [
            {"index": int(i), "confidence": float(c)}
            for i, c in zip(best_indices, best_confidences)
        ]
    }

with open('./config.yaml', 'r') as f:
    config = yaml.safe_load(f)
    risg = RISGMatcher(config)

imgs_path = 'test'
folder_count = sum(1 for name in os.listdir(imgs_path) if os.path.isdir(os.path.join(imgs_path, name)))

*  --Device for matching：cpu


In [7]:
%cd ..

/home/kali/Documents/personal/computer_vision/risg_image_matching


In [8]:
benchmarks = []

for i in range(1, folder_count + 1):
    img_filename0 = f'RISG-image-matching/test/0{i}/pair1.jpg'
    img_filename1 = f'RISG-image-matching/test/0{i}/pair2.jpg'
    img0, img1, mkpts0, mkpts1, conf, main_dir = run_model(risg, img_filename0, img_filename1)

    draw_matches(img0, img1, mkpts0, mkpts1)

    threshold = 0.9
    bench = create_benchmark(conf, i, threshold)
    benchmarks.append(bench)

out_path = "benchmarks.json"
with open(out_path, "w") as jf:
    json.dump(benchmarks, jf, indent=2)

print(f"Saved benchmarks for {len(benchmarks)} image pairs to {out_path}")

Saved benchmarks for 6 image pairs to benchmarks.json
